# Lesson 2: Linear models with CNN features

## Linear model in Keras

* Dense() layers are just linear models, followed by a simple "activation function".
* Example linear model:

In [6]:
import os
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

import utils; reload(utils)
from utils import plots, get_batches

In [17]:
x = np.random.random((60, 2))
y = np.dot(x, [3., 7.]) + 1

In [18]:
x[:5]

array([[ 0.12060898,  0.53678266],
       [ 0.88930682,  0.16812984],
       [ 0.61408849,  0.6553499 ],
       [ 0.72015569,  0.94614384],
       [ 0.12886502,  0.66056675]])

In [19]:
y[:5]

array([ 5.11930555,  4.84482937,  7.42971478,  9.78347392,  6.0105623 ])

Can create a simple linear model (Dense() - with no action) and optimise using stochastic gradient descent, minimising mean squared error (mse):

In [26]:
lm = Sequential([Dense(1, input_shape=(2,))])
lm.compile(optimizer=SGD(lr=0.1), loss='mse')

In [27]:
lm.evaluate(x, y, verbose=0)

34.398860931396484

In [28]:
lm.fit(x, y, nb_epoch=5, batch_size=1)

Epoch 1/5
60/60 [==============================] - 0s - loss: 1.8652     
Epoch 2/5
60/60 [==============================] - 0s - loss: 0.2047     
Epoch 3/5
60/60 [==============================] - 0s - loss: 0.0508     
Epoch 4/5
60/60 [==============================] - 0s - loss: 0.0142     
Epoch 5/5
60/60 [==============================] - 0s - loss: 0.0041     


In [29]:
lm.evaluate(x, y, verbose=0)

0.0025018354101727408

In [30]:
# Note that the weighs are close to the y weights (3, 7) with an intercept of 1.
lm.get_weights()

[array([[ 2.91822338],
        [ 6.85827494]], dtype=float32), array([ 1.09764993], dtype=float32)]

Can use a Dense() layer to convert the 1000 ImageNet predictions into probably of Dog vs Cat, by training a linear model to take 1000 predictions as input and return Dog or Cat as output.

In [3]:
path = "/home/ubuntu/nbs/data/male_female_training_set_20170601/"
model_path = path + 'models'
if not os.path.exists(model_path): os.mkdir(model_path)

In [4]:
batch_size = 64

In [5]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

Approach:

1. Get true labels for every image.
2. Get 1,000 ImageNet category predictions for each image.
3. Feed predictions as input to simple linear model.

In [8]:
val_batches = get_batches(path+'test', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)

Found 668 images belonging to 3 classes.
Found 6450 images belonging to 3 classes.


Can save the preprocessed arrays using bcolz, which also compresses the arrays.

In [10]:
import bcolz

def save_array(fname, arr):
    c = bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]